# Part I. ETL Pipeline for Pre-Processing the Files

#### Importing Python packages 

In [49]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from tabulate import tabulate

#### Creating list of filepaths to process original event csv data files

In [50]:
# checking the current working directory
print(os.getcwd())

# Getting the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Creating a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# Joining the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [52]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [53]:
# checking the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. The Apache Cassandra coding portion of the project. 

## The CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data appears like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">


#### Creating a Cluster

In [54]:
# Setting up a connection to a Cassandra instance on the local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# Setting up a session to establish connection and begin executing queries
session = cluster.connect()

#### Creating Keyspace

In [55]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Setting Keyspace

In [56]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)


## The Apache Cassandra tables below have been created to cater for the following queries:



### 1. The analyst wants to find the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

For this query:  
- the data is partitioned by *session_id* 
- the clustering column *item_in_session* is added to ensure the uniqueness of the PRIMARY KEY.


In [57]:
# Creating the table
table1 = "CREATE TABLE IF NOT EXISTS songs_in_session"
table1 = table1 + "(session_id int, item_in_session int, artist_name text,\
                    song_title text, song_length float,\
                    PRIMARY KEY (session_id,item_in_session))"
try:
    session.execute(table1)
except Exception as e:
    print(e)          

In [58]:
# Setting up the csv file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Inserting values into the songs_in_sessions table
        insert1 = "INSERT INTO songs_in_session (session_id, item_in_session, artist_name, song_title, song_length)"
        insert1 = insert1 + " VALUES (%s, %s, %s, %s, %s)"
         # Setting up the correct order of values and data types
        session.execute(insert1, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Running a SELECT statement to verify that the data have been inserted into songs_in_session table and to answer the business question specified above.

In [59]:
# CQL SELECT statement to run
query1 = "select artist_name, song_title, song_length  FROM songs_in_session WHERE session_id = 338 AND item_in_session = 4"
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)
# Printing the data in a tabular form    
column_names1 = ['artist_name', 'song_title', 'song_length']

df1 = pd.DataFrame(columns = column_names1)    
for row in rows:
    to_append = [row.artist_name, row.song_title, row.song_length]
    df1_length = len(df1)
    df1.loc[df1_length] = to_append  
print (tabulate(df1, showindex=False, headers=df1.columns, tablefmt = 'psql'))

+---------------+---------------------------------+---------------+
| artist_name   | song_title                      |   song_length |
|---------------+---------------------------------+---------------|
| Faithless     | Music Matters (Mark Knight Dub) |       495.307 |
+---------------+---------------------------------+---------------+



### 2. The analyst wants to find the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

For this query:  
- the data is partitioned by *user_id* 
- the clustering columns *session_id* and *item_in_session* are added to ensure the uniqueness of the PRIMARY KEY and to sort the data.


In [60]:
# Creating the table
table2 = "CREATE TABLE IF NOT EXISTS user_sessions"
table2 = table2 + "(user_id int, session_id int, item_in_session int, artist_name text,\
                    song_title text, first_name text, last_name text,\
                    PRIMARY KEY (user_id, session_id,item_in_session))"
try:
    session.execute(table2)
except Exception as e:
    print(e)                 

In [61]:
# Setting up the csv file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Inserting values into the user_sessions table
        insert2 = "INSERT INTO user_sessions (user_id, session_id, item_in_session, artist_name, song_title, first_name, last_name)"
        insert2 = insert2 + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        # Setting up the correct order of values and data types
        session.execute(insert2, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))                 

#### Running a SELECT statement to verify that the data have been inserted into user_sessions table and to answer the business question specified above.

In [62]:
# CQL SELECT statement to run
query2 = "select first_name, last_name, artist_name, song_title FROM user_sessions WHERE user_id = 10 AND session_id = 182"
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)
# Printing the data in a tabular form     
column_names2 = ['user', 'artist_name', 'song_title']

df2 = pd.DataFrame(columns = ['user', 'artist_name', 'song_title'])  

for row in rows:
    to_append = [row.first_name + ' ' + row.last_name, row.artist_name, row.song_title]
    df2_length = len(df2)
    df2.loc[df2_length] = to_append  
print (tabulate(df2, showindex=False, headers=df2.columns, tablefmt = 'psql'))

+-------------+-------------------+------------------------------------------------------+
| user        | artist_name       | song_title                                           |
|-------------+-------------------+------------------------------------------------------|
| Sylvie Cruz | Down To The Bone  | Keep On Keepin' On                                   |
| Sylvie Cruz | Three Drives      | Greece 2000                                          |
| Sylvie Cruz | Sebastien Tellier | Kilometer                                            |
| Sylvie Cruz | Lonnie Gordon     | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |
+-------------+-------------------+------------------------------------------------------+


### 3. The analyst wants to find every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

For this query:  
- the data is partitioned by *song_title* 
- the clustering columns *session_id* and *item_in_session* are added to ensure the uniqueness of the PRIMARY KEY.

In [63]:
# Creating the table
table3 = "CREATE TABLE IF NOT EXISTS song_data"
table3 = table3 + "(song_title text, session_id int, item_in_session int,\
                    first_name text, last_name text,\
                    PRIMARY KEY (song_title, session_id,item_in_session))"
try:
    session.execute(table3)
except Exception as e:
    print(e)    

In [64]:
# Setting up the csv file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Inserting values into the song_data table
        insert3 = "INSERT INTO song_data (song_title, session_id, item_in_session, first_name, last_name)"
        insert3 = insert3 + " VALUES (%s, %s, %s, %s, %s)"
        # Setting up the correct order of values and data types
        session.execute(insert3, (line[9],int(line[8]), int(line[3]), line[1], line[4]))                 

#### Running a SELECT statement to verify that the data have been inserted into song_data table and to answer the business question specified above.

In [65]:
# CQL SELECT statement to run
query3 = "select first_name, last_name FROM song_data WHERE song_title = 'All Hands Against His Own'"
try:
    rows = session.execute(query3)
except Exception as e:
    print(e)
    
# Printing the data in a tabular form     
column_names3 = ['user']

df3 = pd.DataFrame(columns = column_names3)   

for row in rows:
    to_append = [row.first_name + ' ' + row.last_name]
    df3_length = len(df3)
    df3.loc[df3_length] = to_append  
print (tabulate(df3, showindex=False, headers=df3.columns, tablefmt = 'psql'))

+------------------+
| user             |
|------------------|
| Sara Johnson     |
| Jacqueline Lynch |
| Tegan Levine     |
+------------------+


### Dropping the tables before closing out the sessions

In [66]:
query = "DROP TABLE songs_in_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE user_sessions"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE song_data"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [67]:
session.shutdown()
cluster.shutdown()